## Introduction

### Imports

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import json
from pprint import pprint

## Data exploration - Tree

In [ ]:
from argonodes.nodes import Tree

### First, we load the raw JSON data

In [ ]:
filename = "../inputs/2022_MARCH.json"

with open(filename, "r") as jsonfile:
    json_data = json.loads(jsonfile.read())
    
print(str(json_data)[:512] + "...")

In [ ]:
filename = filename.split("/")[-1]

### Then, we create a tree with the data

In [ ]:
tree = Tree(json_data, filename=filename)

### We can explore whatever is inside

In [ ]:
tree

In [ ]:
tree.children[0]

In [ ]:
print(len(tree.get_paths()))
tree.get_paths()

In [ ]:
tree.get_children_from_path("$.timelineObjects[4].activitySegment.activities[1].activityType")

#### `get_children_from_path` supports wildcards and returns every match

In [ ]:
tree.get_children_from_path("$.timelineObjects[4].activitySegment.activities[*].*")

## Data semantics - Model

In [ ]:
from argonodes.models import Model

### We create a new model from an existing Tree

In [ ]:
model = Model(tree, name="Google Geolocation – YEAR_MONTH.json")

### Now, we can see what paths are available in our model

In [ ]:
model.get_paths()

### But more importantly, we can see what attributes are linked to these paths

In [ ]:
import pandas as pd

In [ ]:
columns, liste = model.to_list()
df = pd.DataFrame(data=liste[filename], columns=columns)
df.head()

### It's a bit empty for now... Let's add some info.

In [ ]:
model.set_attributes(
    "$",
    filename,
    description="Model for the list of semantic locations determined by Google. Can be either `placeVisit` or `activitySegment`."
)

In [ ]:
model.set_attributes(
    "$.timelineObjects",
    filename,
    unique=True,
    description="List of semantic locations determined by Google. Can be either `placeVisit` or `activitySegment`."
)

In [ ]:
model.set_attributes(
    "$.timelineObjects[*]",
    filename,
    unique=False,
    description="One of the semantic locations. Can be either `placeVisit` or `activitySegment`."
)

In [ ]:
model.set_attributes(
    "$.timelineObjects[*].placeVisit.location",
    filename,
    descriptiveType="https://schema.org/location",
    unique=False,
    description="Location that Google thinks you visited (latitude, longitude, id, address, name and confidence)."
)

In [ ]:
columns, liste = model.to_list()
df = pd.DataFrame(data=liste[filename], columns=columns)
df.head()

## Data semantics - Apply the model

### Now that we have a working model, let's apply it back to our existing tree

In [ ]:
tree.get_children_from_path("$.timelineObjects[0].placeVisit.location")[0]

In [ ]:
tree.apply(model)

In [ ]:
tree.get_children_from_path("$.timelineObjects[0].placeVisit.location")[0]

## Data filtering - Add filters to the model

### We can filter the model in ordrer to only keep what we need.

In [ ]:
from argonodes.filters import Filter

In [ ]:
placeVisit = Filter(path__startswith="$.timelineObjects[*].placeVisit")
onlyLocation = Filter(descriptiveType__exact="https://schema.org/location")

In [ ]:
placeVisit(model)

In [ ]:
onlyLocation(model)

In [ ]:
model.traversal

## Data concierge - Export and import

### If you have an existing model, you can export it.

In [ ]:
model.export_traversal(filename="my_model.pickle")

### Afterwards it can be imported as well.

In [ ]:
model = Model(name="Google Geolocation – YEAR_MONTH.json")
model

In [ ]:
model.load_traversal(filename="my_model.pickle")

In [ ]:
model.traversal